In [ ]:
import os
import json
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib as mpl

device = torch.device("cpu")

In [ ]:
workdir = os.path.join(
    os.path.dirname(os.getcwd()),
    'lorenz_results/lorenz96-241031-150134',
)

with open(os.path.join(workdir, 'config.json'), 'r') as f:
    cfg = json.load(f)

dim = cfg['dynamics']['dim']
steps = cfg['dynamics']['steps']
dt = cfg['dynamics']['dt']
noise_std = cfg['measurement']['noise_std']

results = np.load(os.path.join(workdir, 'results.npz'))
states = results['states'] # (steps+1, dim)
observations = results['observations'] # (steps+1, dim)
ssls_states = results['assimilated_states'] # (steps+1, nsamples, dim)

ssls_mean = np.mean(ssls_states, axis=1)
guess_init = np.ones((1, dim)) * cfg["dynamics"]["prior_mean"]
ssls_mean = np.concat((guess_init, ssls_mean))

results = np.load(f'../lorenz_results/apf_sigma{noise_std}_nensemble{cfg["train"]["n_train"]}.npz')
apf_states = results['assimilated_states'] # (steps+1, nsamples, dim)
apf_mean = np.mean(apf_states, axis=1)
apf_mean = np.concat((guess_init, apf_mean))

In [ ]:
with open("../asset/Lorenz96_evolution.pkl", "rb") as file:
    states, ssls_mean, apf_mean = pickle.load(file)
mpl.rcdefaults()
mpl.style.use("../configs/mplrc")
mpl.rc("lines", linewidth=1.5, marker='o', markersize=4, markerfacecolor='none')


d1, d2 = 1, 20
markevery = 1
start_steps = 0
stop_steps = 30

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 3.5))
# true trajectory
ax.plot(states[:, d1-1][start_steps+1:stop_steps:markevery], states[:, d2-1][start_steps+1:stop_steps:markevery], color='C0', markevery=markevery)
# ssls trajectory
ax.plot(ssls_mean[:, d1-1][start_steps+1:stop_steps+1:markevery], ssls_mean[:, d2-1][start_steps+1:stop_steps+1:markevery], color='C1', markevery=markevery)
# ssls first step trajectory
ax.plot(ssls_mean[:, d1-1][start_steps:start_steps+2], ssls_mean[:, d2-1][start_steps:start_steps+2], color='C1')
# apf trajectory
ax.plot(apf_mean[:, d1-1][start_steps+1:stop_steps+1:markevery], apf_mean[:, d2-1][start_steps+1:stop_steps+1:markevery], color='C2', markevery=markevery)
# apf first step trajectory
ax.plot(apf_mean[:, d1-1][start_steps:start_steps+2], apf_mean[:, d2-1][start_steps:start_steps+2], color='C2')

# mark init
ax.scatter(states[:, d1-1][start_steps], states[:, d2-1][start_steps], color='C0', marker='*', s=200, zorder=99, facecolor='none')
ax.scatter(ssls_mean[:, d1-1][start_steps], ssls_mean[:, d2-1][start_steps], color='C3', marker='*', s=200, zorder=99, facecolor='none')
ax.set_xlabel(r"$\bf{x}_{1:30," f"{d1}" r"}$")
ax.set_ylabel(r"$\bf{x}_{1:30," f"{d2}" r"}$")
custom_lines = [
    mpl.lines.Line2D([0], [0], color='C0', label='Truth'),
    mpl.lines.Line2D([0], [0], marker='*', color='w', markerfacecolor='none', markeredgecolor='C0', markeredgewidth=1, markersize=10, label='True start'),
    mpl.lines.Line2D([0], [0], color='C1', label='SSLS'),
    mpl.lines.Line2D([0], [0], marker='*', color='w', markerfacecolor='none', markeredgecolor='C3', markeredgewidth=1, markersize=10, label='SSLS/APF start'),
    mpl.lines.Line2D([0], [0], color='C2', label='APF'),
]
ax.legend(handles=custom_lines, bbox_to_anchor=(0.1, 1.0), loc='lower left', ncol=3)
plt.savefig('../asset/Lorenz96_trajectory.pdf', dpi=600, bbox_inches='tight', pad_inches=0.1)